In [29]:
from imblearn.over_sampling._smote.base import SMOTE
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

preprocessed_accidents_df = pd.read_csv('dataset/pre-processed_dataset_no_state.csv')

def change(record):
    if record == 2:
        return 0
    elif record == 3:
        return 1

temp = list(map(change,preprocessed_accidents_df['Severity']))
preprocessed_accidents_df['Severity'] = temp

def change_day(record):
    if record == 0:
        return str('monday')
    elif record == 1:
        return str('tuesday')
    if record == 2:
        return str('wednesday')
    elif record == 3:
        return str('thursday')
    if record == 4:
        return str('friday')
    elif record == 5:
        return str('saturday')
    if record == 6:
        return str('sunday')

temp = list(map(change_day,preprocessed_accidents_df['Start_Time']))
preprocessed_accidents_df['Start_Time'] = temp

for column in preprocessed_accidents_df.columns:
    if column in ['Start_Time']:
      dummies = pd.get_dummies(preprocessed_accidents_df[column],drop_first=False)
      preprocessed_accidents_df = pd.concat([preprocessed_accidents_df,dummies],axis=1)
      preprocessed_accidents_df.drop(column,axis=1,inplace=True)

y = preprocessed_accidents_df['Severity']
X = preprocessed_accidents_df.drop('Severity',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline_estimators = [('scaling',StandardScaler()),('clf',LogisticRegression())]
pipe = Pipeline(pipeline_estimators)



In [30]:
from sklearn.metrics import accuracy_score


pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_pred,y_test)

0.8206280073545712

In [31]:
from collections import Counter

Counter(y_pred)

Counter({0: 152308, 1: 1066})

In [32]:
X_test.loc[[2]]

,Unnamed: 0,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Side,Temperature,Wind_Chill,Humidity,...,Widespread Dust,Wintry Mix,Wintry Mix / Windy,friday,monday,saturday,sunday,thursday,tuesday,wednesday
2,2,39.10266,-84.52468,39.10209,-84.52396,0.055,1,36.0,51.887338,97.0,...,0,0,0,0,1,0,0,0,0,0


In [33]:

pred = pipe.predict(X_test.loc[[2]])

In [34]:
pred[0]

0

In [35]:
import joblib

joblib.dump(pipe, 'completed_model.joblib')

['completed_model.joblib']